In [5]:
import timeatlas as ta
from timeatlas import TimeSeries, Metadata
from timeatlas import models
from pandas import Series, DatetimeIndex
import pandas as pd
import os
from pathlib import Path
import fbprophet

---
# Models

How to create models with TimeAtlas? At first let's create a TimeSeries.

In [7]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
ts

## Grammar

Some definitions:

- `time series` : self-explanatory, i.e. the TimeSeries object
- `horizon` : the duration to predict after the last value of the time series
- `frequency`: the number of values per unit of time. Usually, the frequency is given in Pandas offset aliases (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

```
                                horizon
   |-------------------------|- - - - - -|
                             |||||||||||||
           time series         frequency

```

In [ ]:
m = models.Prophet()

In [ ]:
m.fit(ts)

In [ ]:
Y_hat = m.predict('4 days')

In [ ]:
m.X_train.plot()
Y_hat.plot()

# NN-Models

### MLP Classification

In [1]:
from pandas import Series, DatetimeIndex
from torch.utils.data import DataLoader
from timeatlas import TimeSeriesDataset, TimeSeries
from timeatlas.models.NN import TimeSeriesClassificationDataset

In [2]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
my_series2 = Series([0.0, 1.1, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
ts2 = TimeSeries(my_series2)
ts.label = 1
tsd = TimeSeriesDataset(data=[ts, ts2])

In [3]:
dataset = TimeSeriesClassificationDataset(tsd)
dataloader = DataLoader(dataset, batch_size=1)

dataset.data

array([[0.4, 1. , 0.7, 0.6],
       [0. , 1.1, 0.7, 0.6]])

In [5]:
dataset.min_max_norm()


array([[0.36363636, 0.90909091, 0.63636364, 0.54545455],
       [0.        , 1.        , 0.63636364, 0.54545455]])

In [6]:
dataset.z_score_norm()

array([[-0.74239143,  1.13312376,  0.19536617, -0.1172197 ],
       [-1.9927349 ,  1.44570963,  0.19536617, -0.1172197 ]])

In [5]:
for i in dataloader:
    print(i)

[tensor([[0.4000, 1.0000, 0.7000, 0.6000]], dtype=torch.float64), tensor([1])]
[tensor([[0.4000, 1.0000, 0.7000, 0.6000]], dtype=torch.float64), tensor([1])]


### LSTM Prediction

In [1]:
from pandas import Series, DatetimeIndex
from torch.utils.data import DataLoader
from timeatlas import TimeSeriesDataset, TimeSeries
from timeatlas.models import LSTMModel
from timeatlas.models.NN.models.LSTM import LSTM
from timeatlas.models.NN import TimeSeriesPredictionDataset

import torch.nn as nn
import torch

import numpy as np
import datetime

In [2]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
tsd = TimeSeriesDataset(data=[ts, ts])

In [3]:
dataset = TimeSeriesPredictionDataset(data=tsd, n=3)
dataloader = DataLoader(dataset, batch_size=2)
dataset.data

array([[0.4, 1. , 0.7],
       [0.4, 1. , 0.7]])

In [4]:
model = LSTM(n_features=1, n_hidden=32, seq_len=3, n_layers=2, dropout=0.1).double()
NN = LSTMModel(model=model, epochs=100, optimizer=torch.optim.Adam, loss_function=nn.MSELoss(), lr=0.001)

In [5]:
model

LSTM(
  (lstm): LSTM(1, 32, num_layers=2, dropout=0.1)
  (linear): Linear(in_features=32, out_features=1, bias=True)
)

In [6]:
NN.fit(dataloader)

/Users/lorenz/.local/share/virtualenvs/timeatlas-kzsAXcI_/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:   1 loss: 0.17084485
epoch:  26 loss: 0.04346432
epoch:  51 loss: 0.00215728
epoch:  76 loss: 0.00065843
epoch:  99 loss: 0.0001986753
